In [1]:
import open3d as o3d
import trimesh
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# ---------- USER SETTINGS ----------
stl_path = "BruceAssembly.stl"
dae_path = "BruceAssembly.dae"

In [3]:
scale_ratio = 0.03382
target_faces = 147295  # set None to skip

mesh = o3d.io.read_triangle_mesh(stl_path)

# Clean up common STL issues
mesh.remove_duplicated_vertices()
mesh.remove_duplicated_triangles()
mesh.remove_degenerate_triangles()
mesh.remove_non_manifold_edges()

# Uniform scale about origin
mesh.scale(scale_ratio, center=(0.0, 0.0, 0.0))

# Optional decimation
if target_faces is not None:
    mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=int(target_faces))

# Recompute normals (critical for not-black shading)
mesh.compute_vertex_normals()

# Report
V = np.asarray(mesh.vertices)
F = np.asarray(mesh.triangles)
print("Vertices:", V.shape[0])
print("Triangles:", F.shape[0])
print("BBox extents:", mesh.get_axis_aligned_bounding_box().get_extent())


Vertices: 72656
Triangles: 147295
BBox extents: [18.01259878 13.0966278  16.36374889]


In [4]:
import collada
from collada import material, geometry, scene, source

V = np.asarray(mesh.vertices, dtype=np.float32)
F = np.asarray(mesh.triangles, dtype=np.int32)

# Use Open3D vertex normals if present, else compute per-vertex from faces
N = np.asarray(mesh.vertex_normals, dtype=np.float32)
if N.shape[0] != V.shape[0]:
    # Fallback, should not happen if compute_vertex_normals ran
    N = np.zeros_like(V, dtype=np.float32)

# Collada document
dae = collada.Collada()
dae.assetInfo.unitname = "meter"
dae.assetInfo.unitmeter = 1.0
dae.assetInfo.upaxis = collada.asset.UP_AXIS.Z_UP

# Simple gray material so it does not render black
effect = material.Effect(
    "effect0",
    [],
    "phong",
    diffuse=(0.75, 0.75, 0.75, 1.0),
    specular=(0.15, 0.15, 0.15, 1.0),
    shininess=20.0
)
mat = material.Material("material0", "GrayMaterial", effect)
dae.effects.append(effect)
dae.materials.append(mat)

# Geometry sources
vert_src = source.FloatSource("verts-array", V, ("X", "Y", "Z"))
norm_src = source.FloatSource("normals-array", N, ("X", "Y", "Z"))

geom = geometry.Geometry(dae, "geometry0", "mesh", [vert_src, norm_src])

# Create triangle set with vertex and normal indices
# We map normals 1-to-1 with vertices, so normal indices equal vertex indices.
tri_indices = np.empty((F.shape[0], 6), dtype=np.int32)
tri_indices[:, 0] = F[:, 0]  # v0
tri_indices[:, 1] = F[:, 0]  # n0
tri_indices[:, 2] = F[:, 1]  # v1
tri_indices[:, 3] = F[:, 1]  # n1
tri_indices[:, 4] = F[:, 2]  # v2
tri_indices[:, 5] = F[:, 2]  # n2

input_list = source.InputList()
input_list.addInput(0, "VERTEX", "#verts-array")
input_list.addInput(1, "NORMAL", "#normals-array")

triset = geom.createTriangleSet(tri_indices, input_list, "material0")
geom.primitives.append(triset)
dae.geometries.append(geom)

# Scene graph
matnode = scene.MaterialNode("material0", mat, inputs=[])
geomnode = scene.GeometryNode(geom, [matnode])
node = scene.Node("node0", children=[geomnode])

myscene = scene.Scene("scene0", [node])
dae.scenes.append(myscene)
dae.scene = myscene

dae.write(dae_path)
print("Wrote:", dae_path)


Wrote: BruceAssembly.dae
